In [2]:
import pandas as pd
stpo = pd.read_csv(r'C:/Users/eheima/VSC/RawMaterial/stpo.csv', sep =";", dtype = str)
mast = pd.read_csv(r'C:/Users/eheima/VSC/RawMaterial/mast.csv', sep =";",dtype = str)
stas = pd.read_csv(r'C:/Users/eheima/VSC/RawMaterial/stas.csv', sep =";", dtype = str)

In [3]:

mast['STLAL'] = pd.to_numeric(mast['STLAL'],errors = 'coerce')
mast = mast[mast['STLAL'].notna()]
mast = mast.sort_values(by = ['STLAL'], ascending = False)
mast['Merged_matnr_werks'] = mast['MATNR'] + mast['WERKS']
mast = mast.drop_duplicates('Merged_matnr_werks')



mast = mast.set_index('Merged_matnr_werks', append=False,drop = False)

mast['STLAL'] =mast['STLAL'].astype(int)
mast['STLAL'] =mast['STLAL'].astype(str)

stas['Merged_stlnr_stlal'] = stas['STLNR'] + stas['STLAL']
stas = stas.set_index('Merged_stlnr_stlal', append = False, drop = False)
is_not_x = stas['LKENZ']!='X'
stas = stas[is_not_x]
is_not_X_stpo = stpo['LKENZ']!='X'
stpo = stpo[is_not_X_stpo]
stpo = pd.DataFrame(stpo)

def find_stpo_rows (id, stas, stpo):
    row = mast.loc[id]
    matnr = row['MATNR']
    stlnr = row['STLNR'] 
    stlal = row['STLAL']
    lookup_value = stlnr + stlal
    
    lookuprows = pd.DataFrame(stas.loc[lookup_value])


    lookuprows['Merged_stlnr_stlkn'] = lookuprows['STLNR']+lookuprows['STLKN']
    stpo_lookup = lookuprows['Merged_stlnr_stlkn']

    stpo['Merged_stlnr_stlkn'] = stpo['STLNR'] + stpo['STLKN']
    stpo  =  stpo.set_index('Merged_stlnr_stlkn', append = False, drop = False)

    stpo_selected = stpo.loc[stpo_lookup]
    stpo_selected = pd.DataFrame(stpo_selected)
    stpo_selected['STPOZ'] = stpo_selected['STPOZ'].astype(int)
    stpo_selected = stpo_selected.sort_values(by = ['STPOZ'], ascending = False)
    stpo_selected = stpo_selected.drop_duplicates(subset = 'POSNR')
    bom_items = stpo_selected['IDNRK'].tolist()
    result = {matnr: bom_items}
    
    return result
        

#mast_1 =pd.DataFrame( mast.set_index('MATNR', append = False, drop = False))

In [4]:
import concurrent.futures
dict_all = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    
    future_to_url = {executor.submit(find_stpo_rows,id,stas,stpo):id for id in mast['Merged_matnr_werks']}
 
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            dict_all.update(future.result())
        except:
            pass

In [174]:
print(len(dict_all))
df = pd.DataFrame([dict_all]).T

362


In [175]:
df.index = df.index.set_names('level 1')
df.columns = ['level 2']
df = df.reset_index()

In [176]:
df = df.explode('level 2')
df= df.set_index('level 2', append = False, drop= False)

In [178]:
#Functions for lopping
def add_column (df_1, x):
    
    for i in df_1[f'level {x-1}']:
        
        temp_df = df_1[df_1['level 1']==i]
    
        if len(df_1[df_1['level 1']==i].index!=0):

            ls = temp_df['level 2'].tolist()
            
            list_to_string = ','.join(map(str,ls))
            df_1= df_1.set_index(f'level {x-1}', append = False, drop= False)
            try:
                df_1.loc[i,f'level {x}'] =list_to_string
            except:
                pass
    df_1[f'level {x}'] = df_1[f'level {x}'].str.split(",")
    df_1 = df_1.explode(f'level {x}')
    return df_1

In [177]:
df = add_column(df,3)

UnboundLocalError: local variable 'df' referenced before assignment

In [179]:
x=2
while df[f'level {x}'].isnull().all()!=True:
    x = x+1
    try:
        df = add_column(df,x)
    except:
        break

In [118]:
lis = []
for i in df['level 2']:
    
    temp_df = df[df['level 1']==i]
   
    if len(df[df['level 1']==i].index!=0):
        lis.append(i)
        ls = temp_df['level 2'].tolist()
        
        doubleList =[]
        for j in ls:
            doubleList.append(f"{j}")
        list_to_string = ','.join(map(str,ls))

        try:
            df.loc[i,'level 3'] =list_to_string
        except:
            pass
print(list(dict.fromkeys(lis)))

['000000000000710366', '000000000000712693', '000000000000042561', '000000000000711799', '000000000000042532', '000000000000042531', '000000000000712173', '000000000000710377', '000000000000042513', '000000000000042562', '000000000000042237', '000000000000710190', '000000000000710119', '000000000000713344', '000000000000712128', '000000000000712120', '000000000000042522', '000000000000042220', '000000000000042528', '000000000000042555', '000000000000042537', '000000000000042565', '000000000000713347', '000000000000713346', '000000000000042564', '000000000000042119', '000000000000042529', '000000000000710229', '000000000000710233', '000000000000042227', '000000000000710402', '000000000000042504', '000000000000042503', '000000000000042512', '000000000000042509', '000000000000042518', '000000000000042515', '000000000000042566', '000000000000713637', '000000000000713619', '000000000000713289', '000000000000042523', '000000000000042514', '000000000000713422', '000000000000042535', '00000000

In [119]:
df['level 3'] = df['level 3'].str.split(",")
df = df.explode('level 3')

In [143]:
df.to_csv("C:/Users/eheima/test_raw.csv")